In [3]:
# importing the necessary libraries
import urllib.request
import re
from importlib.metadata import version
import tiktoken

In [4]:
# # ---- Reading the text file from the URL of the text file ----
# url = ("https://raw.githubusercontent.com/rasbt/"
#       "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
#       "the-verdict.txt")
# file_path = "the-verdict.txt"
# urllib.request.urlretrieve(url, file_path)

# with open("the-verdict.txt", "r", encoding="utf-8") as f:
#    raw_text = f.read()
# print("Total number of character:", len(raw_text))
# # print(raw_text[:99])

In [5]:
# # ---- Preprocessing the text to split the text into tokens ----
# preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
# preprocessed = [item.strip() for item in preprocessed if item.strip()]
# print(len(preprocessed))
# # print(preprocessed[:30])

In [6]:
# # ---- Building a vocabulary from the preprocessed (tokenised) text ----
# all_words = sorted(set(preprocessed))
# all_words.extend(["<|endoftext|>", "<|unk|>"])
# vocab = {token:integer for integer,token in enumerate(all_words)}

# class SimpleTokenizerV2:
#       def __init__(self, vocab):
#             self.str_to_int = vocab
#             self.int_to_str = {i:s for s,i in vocab.items()}

#       def encode(self, text):
#             preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
#             preprocessed = [item.strip() for item in preprocessed if item.strip()]
#             preprocessed = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed]

#             ids = [self.str_to_int[s] for s in preprocessed]
#             return ids

#       def decode(self, ids):
#             text = " ".join([self.int_to_str[i] for i in ids])
#             text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
#             return text

# # tokenizer = SimpleTokenizerV1(vocab)
# # text =  """"It's the last he painted, you know," Mrs. Gisburn said with pardonable pride."""
# # ids = tokenizer.encode(text)
# # print(tokenizer.decode(ids))

# text1 = "Hello, do you like tea?"
# text2 = "In the sunlit terraces of the palace."
# text3 = "painted."
# text4 = "pardonable."
# text5 = "lmao."
# text = " <|endoftext|> ".join((text1, text2, text3, text4, text5))
# # print(text)
# tokenizer = SimpleTokenizerV2(vocab)
# # print(tokenizer.encode(text))
# # print(tokenizer.decode(tokenizer.encode(text)))

In [7]:
# --- Using tiktoken to implement BPE ----

# print("tiktoken version:", version("tiktoken"))
# tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")
# text = ("Hello, do you like tea? <|endoftext|> In the sunlit terraces"
#         "of Akwirw ier." )
# integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"}, disallowed_special=())
# print(integers)
# strings = tokenizer.decode(integers)
# print(strings)

# # ---- create input-target pairs ----
# with open("the-verdict.txt", "r", encoding="utf-8") as f:
#       raw_text = f.read()

# enc_text = tokenizer.encode(raw_text)
# print(len(enc_text))

# enc_sample = enc_text[50:]

# context_size = 4
# x = enc_sample[:context_size]
# y = enc_sample[1:context_size+1]
# print(f"x: {x}")
# print(f"y:      {y}")

# for i in range(1, context_size+1):
#       context = enc_sample[:i]
#       desired = enc_sample[i]
#       print(context, "---->", desired)

# for i in range(1, context_size+1):
#       context = enc_sample[:i]
#       desired = enc_sample[i]
#       print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

In [8]:
# using PyTorch dataset

import torch
from torch.utils.data import Dataset, DataLoader
# torch.__version__

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []
        token_ids = tokenizer.encode(txt)   

        for i in range(0, len(token_ids) - max_length, stride):    
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):   
        return len(self.input_ids)
    
    def __getitem__(self, idx):        
        return self.input_ids[idx], self.target_ids[idx]


In [9]:
# creating PyToch DataLoader using the GPTDatasetV1 class along with BPE tiktoken tokenizer 

def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride = 128, shuffle = True, drop_last = True, num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")                        
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)  
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,    
        num_workers=num_workers    
    )
    return dataloader

In [10]:
# basic example to use the dataloader

# sometimes, takes a long time to execute (more than 1 minute, idk why)

with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

dataloader = create_dataloader_v1(
    raw_text, batch_size=4, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)     
first_batch = next(data_iter)
print(first_batch)


[tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257]]), tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922]])]


In [11]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

max_length = 4
dataloader = create_dataloader_v1(
   raw_text, batch_size=8, max_length=max_length,
  stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])
torch.Size([8, 4, 256])


In [13]:
# For a GPT model’s absolute embedding approach, we just need to create another
#  embedding layer that has the same embedding dimension as the token_embedding_
#  layer:  The input to the pos_embeddings is usually a placeholder vector torch.arange(con
# text_length), which contains a sequence of numbers 0, 1, ..., up to the maximum
#  input length –1. The context_length is a variable that represents the supported input
#  size of the LLM. Here,
# For a GPT model’s  we choose it similar to the maximum length of the input text.
#  In practice, input text can be longer than the supported context length, in which case
#  we have to truncate the text.

context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
print(pos_embedding_layer)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)
print("\ntoken embeddings\n", token_embeddings)
print("\npos embeddings\n", pos_embeddings)
print("\ninput embeddings\n", input_embeddings)

# we now have the input embeddings that are used as input for the main LLM layers

Embedding(4, 256)
torch.Size([4, 256])
torch.Size([8, 4, 256])

token embeddings
 tensor([[[-1.3940,  0.3517,  0.9450,  ..., -0.1821,  0.0900, -0.8099],
         [ 0.9372, -1.5136,  1.0291,  ..., -0.0958, -0.5204, -0.0281],
         [ 1.5280,  0.1011,  0.4999,  ...,  1.0641, -1.8279, -0.5470],
         [ 1.2461, -0.0260, -1.3018,  ...,  0.1149, -1.5188, -0.0327]],

        [[ 0.9334,  0.3306,  0.3520,  ...,  0.1425,  1.4388, -0.3138],
         [ 2.2529, -1.2778, -0.3451,  ...,  0.4945, -0.9861,  1.0731],
         [-0.0956, -0.8321, -0.8844,  ..., -0.4188,  0.5174, -1.0467],
         [ 0.6763,  0.4832,  0.6521,  ..., -2.3425,  1.6773,  0.7270]],

        [[ 0.9394, -0.3431, -2.4999,  ...,  2.0133, -0.6048,  0.9944],
         [-0.7053, -0.5862,  1.2643,  ...,  0.5512, -1.3381, -1.1105],
         [-0.5063, -1.3788,  0.2553,  ...,  0.1776,  0.9269, -0.8399],
         [ 1.3182,  1.0709, -0.0928,  ..., -1.0938, -1.5077,  0.6416]],

        ...,

        [[ 0.1704, -0.8800,  2.2553,  ..., -1.